# LSTM Language Model

Today we are going to build a language model using pytorch.  

Instead of using word and character tokens, we are going to use **subwords**. Words can be furthur broken down into subword units which usually have meaning.  
Example:
basketball => basket@@ ball  
everyday => every@@ day

'@@ ' here denotes continuation of a word.

**Byte Pair Encoding (BPE)** creates common tokens that can be used to spilt out-of-vocabulary words.

Install subword-nmt for BPE tokenization:
    `pip install subword-nmt`

## Learn BPE tokens
First, we are going to learn 10000 most common units from splitting OOV words using training data.

In [ ]:
# Learn a bpe vocabulary using subword-nmt 
!subword-nmt learn-bpe -s 10000 < data/ptb.train.txt > data/codes.bpe


## Apply BEP 
Apply BPE to the train, test and valid data.

In [ ]:
# Apply bpe to the data files
!subword-nmt apply-bpe -c data/codes.bpe < data/ptb.train.txt > data/ptb.train.bpe.txt
!subword-nmt apply-bpe -c data/codes.bpe < data/ptb.test.txt > data/ptb.test.bpe.txt
!subword-nmt apply-bpe -c data/codes.bpe < data/ptb.valid.txt > data/ptb.valid.bpe.txt


Let's check the bpe coded file we just created!

In [1]:
!head data/ptb.train.bpe.txt

 a@@ er ban@@ kno@@ te ber@@ l@@ it@@ z cal@@ low@@ ay cen@@ trust clu@@ et@@ t fro@@ m@@ stein g@@ it@@ an@@ o gu@@ ter@@ man hy@@ dro@@ -@@ quebec ip@@ o k@@ ia me@@ mo@@ te@@ c m@@ l@@ x na@@ h@@ b p@@ un@@ ts r@@ ake reg@@ att@@ a ru@@ ben@@ s si@@ m sn@@ ac@@ k-@@ food s@@ san@@ gy@@ ong sw@@ ap@@ o w@@ ach@@ ter 
 pi@@ er@@ re <unk> N years old will join the board as a non@@ executive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 ru@@ dol@@ ph <unk> N years old and former chairman of consolidated gold fields plc was named a non@@ executive director of this british industrial conglomerate 
 a form of asbestos once used to make k@@ ent cigarette fil@@ ters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 
 the asbestos fi@@ ber <unk> is unusually <unk> once it ent@@ ers the <unk> with even brief expo@@ sures to it causing symptoms that show up decades later researcher

Now that we have created the data in subword format, we can start writing code for our language model!

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import pickle as pkl
import math
import random
import numpy as np
random.seed(1111)

In [3]:
import pickle as pkl

# Same as previous labs
def tokenize_dataset(dataset): 
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    with open(dataset, 'r') as dataset_file:
        for sample in dataset_file:
            tokens = sample.strip().split() + ['</s>']
            #token_dataset.append(tokens)
            all_tokens += tokens

    return all_tokens

val_data = 'data/ptb.valid.bpe.txt'
test_data = 'data/ptb.test.bpe.txt'
train_data = 'data/ptb.train.bpe.txt'

#print ("Tokenizing val data")
#val_data_tokens = tokenize_dataset(val_data)
#pkl.dump(val_data_tokens, open("val_bpe_tokens.p", "wb"))

# test set tokens
#print ("Tokenizing test data")
#test_data_tokens = tokenize_dataset(test_data)
#pkl.dump(test_data_tokens, open("test_bpe_tokens.p", "wb"))

# train set tokens
#print ("Tokenizing train data")
#train_data_tokens = tokenize_dataset(train_data)
#pkl.dump(train_data_tokens, open("train_bpe_tokens.p", "wb"))

In [4]:
# Then, load preprocessed train, val and test datasets
train_data_tokens = pkl.load(open("train_bpe_tokens.p", "rb"))
val_data_tokens = pkl.load(open("val_bpe_tokens.p", "rb"))
test_data_tokens = pkl.load(open("test_bpe_tokens.p", "rb"))

print ("Train dataset size is {}".format(len(train_data_tokens)))
print ("Val dataset size is {}".format(len(val_data_tokens)))
print ("Test dataset size is {}".format(len(test_data_tokens)))


Train dataset size is 973446
Val dataset size is 78354
Test dataset size is 85981


In [5]:
from collections import Counter

# same as previous labs
max_vocab_size = 20000
def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(0,len(vocab)))) 
    return token2id, id2token

token2id, id2token = build_vocab(train_data_tokens)

In [6]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for token in tokens_data:
        token_id = token2id[token] if token in token2id else token2id['<unk>'] 
        indices_data.append(token_id)
    return indices_data

train_data_indices = torch.LongTensor(token2index_dataset(train_data_tokens))
val_data_indices = torch.LongTensor(token2index_dataset(val_data_tokens))
test_data_indices = torch.LongTensor(token2index_dataset(test_data_tokens))

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 973446
Val dataset size is 78354
Test dataset size is 85981


In [7]:
# We are going to learn the language model for the whole training corpus
# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.

def batchify(data, bsz, random_start_idx=False):
    # calculate total number of batches that fit cleanly
    nbatch = data.size(0) // bsz
    if random_start_idx:
        start_idx = random.randint(0, data.size(0) % bsz - 1)
    else:
        start_idx = 0
        
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    # Nice thing about this: 
    # u don't need to pad since every sequence now has same length
    data = data.narrow(0, start_idx, nbatch * bsz)
    
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data


In [8]:
eval_batch_size = 32
val_data = batchify(val_data_indices, eval_batch_size)
test_data = batchify(test_data_indices, eval_batch_size)

In [9]:
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, vocab_size)

        self.init_weights()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.num_layers, bsz, self.hidden_size),
                    weight.new_zeros(self.num_layers, bsz, self.hidden_size))

In [10]:
embed_size = 200
hidden_size = 400
num_layers = 2
num_epochs = 10
batch_size = 32
lr = 0.1
dropout = 0.3
max_seq_len = 35
vocab_size = len(token2id)
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout)

In [11]:
# get_batch subdivides the source data into chunks of max_seq_len.
# If source is equal to the example output of the batchify function, with
# max_seq_len = 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘

def get_batch(source, i, max_seq_len):
    seq_len = min(max_seq_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


In [13]:
clip = 0.3
log_interval = 200

def repackage_hidden(h):
    """
        Wraps hidden states in new Tensors, to detach them from their history.
    """
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
def train():
    model.train()
    total_loss = 0.
    hidden = model.init_hidden(batch_size)
    
    # We shuffle train data every epoch
    train_data = batchify(train_data_indices, batch_size, random_start_idx=True)
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, max_seq_len)):
        data, targets = get_batch(train_data, i, max_seq_len)
        
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch %log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // max_seq_len, lr,
                cur_loss, math.exp(cur_loss)))
            total_loss = 0


In [ ]:
# perplexity evaluation for a given corpus
def evaluate(data_source, max_seq_len, eval_batch_size=32):
    model.eval()
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, max_seq_len):
            data, targets = get_batch(data_source, i, max_seq_len)
            
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, vocab_size)
            
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / len(data_source)

In [14]:
import numpy as np

best_val_loss = np.inf
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1, num_epochs+1):
    train()
    val_loss = evaluate(val_data, max_seq_len)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, 
                                           val_loss, math.exp(val_loss)))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open('model.pt', 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0


## Load the pretrained model

It's gonna take a while to train this so we provide the pre-trained file in the zip folder. 
Let's load it!

In [15]:
with open('model.pt', 'rb') as f:
    model = torch.load(f)

/Users/phumon/py3torch/lib/python3.6/site-packages/torch/serialization.py:333: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


### Let's compare the perplexity on the test set!

Evaluate the perplexity of pre-trained model on test set

In [16]:
test_loss = evaluate(test_data, max_seq_len)
print("test perplexity: ", math.exp(test_loss))


test perplexity:  157.99914538908837


### Let's check the perplexity on the test set using pre-trained KenLM!

Load the pre-train KenLM model.

In [18]:
import kenlm

# calculate perplexity for KenLM
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    ppl_list = []
    for sent in sentences:
        words = sent.strip().split()
        score = lm.score(sent, bos=False, eos=False)
        word_count = len(words)
        if word_count <=0:
            continue
        total_wc += word_count
        total_nll += score
        sent_ppl = 10**(-score/word_count)
        ppl_list.append((sent, sent_ppl))
    ppl = 10**-(total_nll/total_wc)
    return ppl, ppl_list

with open('data/ptb.test.nounk.txt', 'r') as f:
    sentences = [sent.strip() for sent in f]
    
kenlm_model = kenlm.LanguageModel('ptb_lm_2gram.arpa')
kenlm_ppl, sent_ppl_list = get_ppl(kenlm_model, sentences)


In [24]:
print("total test ppl (KenLM): ", kenlm_ppl)

total test ppl (KenLM):  260.13758257335195


In [20]:
sent_ppl_list[0:5]

[("no it was n't black monday", 244.2144113575105),
 ("but while the new york stock exchange did n't fall apart friday as the dow jones industrial average plunged N points most of it in the final hour it barely managed to stay this side of chaos",
  98.82167725339345),
 ('some circuit breakers installed after the october N crash failed their first test traders say unable to cool the selling panic in both stocks and futures',
  437.0089302511858),
 ("the N stock specialist firms on the big board floor the buyers and sellers of last resort who were criticized after the N crash once again could n't handle the selling pressure",
  208.62356173241537),
 ('big investment banks refused to step up to the plate to support the beleaguered floor traders by buying big blocks of stock traders say',
  189.5969019138085)]

In [21]:
# for getting the perplexity of each sentence in test set
def test(sent_list):
    ppl_list = []
    for sent in sent_list:
        tokens = token2index_dataset(sent.strip().split())
        test_sent_idx = batchify(torch.LongTensor([tokens]), 1)
        loss = evaluate(test_sent_idx, len(tokens), 1)
        ppl_list.append((sent, math.exp(loss)))
    return ppl_list


## Score sentences with RNN_LM

In [22]:
sent_list = ['dividend yields have been bolstered by stock declines', \
             'stock bolstered declines dividend by yields have been', \
             'artificial neural networks are computing systems vaguely inspired by the biological neural networks']
test(sent_list)


[('dividend yields have been bolstered by stock declines', 223.58090727154595),
 ('stock bolstered declines dividend by yields have been', 799.5039174923857),
 ('artificial neural networks are computing systems vaguely inspired by the biological neural networks',
  210.4940782094278)]

## Score the same list of sentences with KenLM

In [25]:
total_ppl, sent_list_ppl = get_ppl(kenlm_model, sent_list)
sent_list_ppl


[('dividend yields have been bolstered by stock declines', 466.40009558112047),
 ('stock bolstered declines dividend by yields have been', 1818.5723239644926),
 ('artificial neural networks are computing systems vaguely inspired by the biological neural networks',
  9918.597743710336)]

We can see that **RNN LM can generalize well and assign lower perplexity** of grammatically correct, out-of-domain sentence, which is not the case with KenLM.

# Exercises

1. Find the perplexity of all sentences in test set using RNN_LM and KenLM. Compare the 10 sentences with lowest and highest perplexity produced by each model. Analyze what kind of sentences are preferred by each model.

2. Train the character level language model and compare the performance.  

2. Create an autocomplete function using the pretrained language model. Example, given a partial sentence, predict the next word.  

3. What is the perplexity if your language model always output uniform distribution, i.e your language model assigns the equal probability to all the tokens in the vocabulary.  

4. Build a convolutional language model. (Reference: https://arxiv.org/abs/1612.08083)